# Recommending Colleges

Sally visits Rice University in Houston, TX and falls in love with the school. What other schools should she apply to? In this case study, we will use the College Scorecard (https://collegescorecard.ed.gov/) data to make recommendations.

# Question 0

Read in the data from https://datasci112.stanford.edu/data/college_attributes.csv, and find Rice in this data set. What row is it?

In [76]:
# YOUR CODE HERE
import pandas as pd
df = pd.read_csv("https://datasci112.stanford.edu/data/college_attributes.csv")

df[df["Institution"] == "Rice University"] # row #1432

Institution     City State  AdmissionRate  Undergraduates  \
1432  Rice University  Houston    TX         0.0948          4240.0   

                                 CarnegieClassification          Ownership  \
1432  Doctoral Universities: Very High Research Acti...  Private nonprofit   

      PCIP01  PCIP03  PCIP04  ...  PCIP44  PCIP45  PCIP46  PCIP47  PCIP48  \
1432     0.0  0.0081  0.0332  ...  0.0161  0.1238     0.0     0.0     0.0   

      PCIP49  PCIP50  PCIP51  PCIP52  PCIP54  
1432     0.0  0.0386  0.0117  0.0018  0.0224  

[1 rows x 45 columns]

# Question 1

Based on only the admission rate and the number of undergraduates, what are the 5 schools that are most similar to Rice?

In [77]:
# YOUR CODE HERE
features = ["Undergraduates", "AdmissionRate"]

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances

scaler = StandardScaler()
scaler.fit(df[features])
standardized = scaler.transform(df[features])

distances = pairwise_distances(standardized[[1432]], standardized)

indices = distances.argsort()[0][1:6] # First one is Rice University
df.loc[indices][["Institution"]]


Institution
548      United States Naval Academy
993   United States Military Academy
172  United States Air Force Academy
791                Dartmouth College
850                  Barnard College

# Question 2
You might be surprised at some of the schools that were found to be "similar" to Rice just based on admission rate and undergraduate size. Clearly, we need to take some categorical variables into account.

Before we do the full analysis, let's first create a dummy encoding of the two categorical variables:

the Carnegie classification (https://en.wikipedia.org/wiki/Carnegie_Classification_of_Institutions_of_Higher_Education)
the ownership (public, private, etc.)
Calculate the sum over the rows (i.e., the sum of each column). What does this represent?

In [78]:
# YOUR CODE HERE

cats = ["CarnegieClassification", "Ownership"]
dummies = pd.get_dummies(df[cats], prefix=["CarnegieClassification", "Ownership"])

col_sums = dummies.sum(axis=0).sort_values(ascending=False)

col_sums

Ownership_Private nonprofit                                                                              1186
Ownership_Public                                                                                          597
CarnegieClassification_Master's Colleges & Universities: Larger Programs                                  277
CarnegieClassification_Baccalaureate Colleges: Diverse Fields                                             228
CarnegieClassification_Baccalaureate Colleges: Arts & Sciences Focus                                      210
Ownership_Private for-profit                                                                              174
CarnegieClassification_Doctoral/Professional Universities                                                 160
CarnegieClassification_Master's Colleges & Universities: Medium Programs                                  157
CarnegieClassification_Not applicable                                                                     155
CarnegieCl

YOUR INTERPRETATION HERE

Each value in col_sums is the number of schools that belong to that dummy category (i.e., how many schools have that Carnegie class or that ownership type).



# Question 3
Now let's do the full analysis.

Considering on the admission rate, the number of undergraduates, the Carnegie classification of the type of school, and the ownership (public, private, etc.), what are the 5 schools that are most similar to Rice?

In [71]:
# YOUR CODE HERE

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

enc = make_column_transformer(
    (OneHotEncoder(), cats), # Only categorical features comes from Question 2
    (StandardScaler(), features), # Only quantitative features comes from Question 1
    remainder="drop"
)

enc.fit(df[cats + features])
mixed_standardized = enc.transform(df[cats + features])

distances = pairwise_distances(mixed_standardized[[1432]], mixed_standardized)

indices = distances.argsort()[0][1:6] # First one is Rice University
df.loc[indices][["Institution"]]


Institution
791                      Dartmouth College
534               Johns Hopkins University
594  Massachusetts Institute of Technology
819                   Princeton University
613                       Tufts University

# Question 4
The columns whose names begin with "PCIP" contain the proportions of students at each school studying various fields (e.g., Engineering, Psychology). Each field is represented by a two-digit code called the CIP code.

What schools are most similar to Rice, if we only consider the proportions of students studying various fields?

In [72]:
# YOUR CODE HERE
pcip_cols = df.filter(regex="^PCIP").columns.tolist() # They are all quantitative


scaler = StandardScaler()
scaler.fit(df[pcip_cols])
pcip_standardized = scaler.transform(df[pcip_cols])

distances = pairwise_distances(pcip_standardized[[1432]], pcip_standardized)

indices = distances.argsort()[0][1:6] # First one is Rice University
df.loc[indices][["Institution"]]


Institution
650          University of Michigan-Ann Arbor
944                   University of Rochester
325   University of Illinois Urbana-Champaign
681       University of Minnesota-Twin Cities
1081          Case Western Reserve University